This tutorial is refered to [this](https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy).

## A Data Science Framework
1. Define the Problem
    * Problems before requirements, requirements before solutions, solutions before design, and design before technology.
2. Gather the Data
3. Prepare Data for Consumption